# 📝 Litsearch summerization

**Topic:** 
In this part we are retrieving the data from OpenAlex and store them in a ChromaDB.

## First extraction of the OpenAlex

We are going to extract data from the https://openalex.org/ with the opensource API. Data available on openalex contain scholarly papers, authors, institutions, and more. We aim to pull all open access papers to create sentence embeddings and store them in our ChromaDB database. Using the API and as suggested by the API wepage we will perform the polite pool. 
Note: There are no daily rate limits, however there is a burst rate limit of 10 requests per second. 

**Note:** This is just the intital pull of the data, we are going to use in our app

## Imports

In [1]:
import pandas as pd 
import requests
import json
import math

import os

## Instantiate the function for the API pull from OpenAlex

We are going to extract data from the https://openalex.org/ with the opensource API. Data available on openalex contain scholarly papers, authors, institutions, and more. We aim to extract information in particular on authors, when they started their path in career, ended it and switched to industry. We also extract information on the number of citations for each publications (over the years), as well as when and from to where researchers switched.

# Insert into Chromadb function

In [5]:
relevant_papers = []

In [6]:
def save_to_chroma(data):
    for entry in data:
        try:
            if entry["primary_location"]["pdf_url"] != None and entry["title"] != None and entry["publication_year"] != None and entry["primary_location"]["source"]["display_name"] != None:
                # check whether given author has no name
                current_authors = [i["author"]["display_name"] for i in entry["authorships"]]
                if None in current_authors:
                    pass
                else:
                    entry_for_list = {
                        "title": entry["title"],
                        "url": entry["primary_location"]["pdf_url"],
                        "year": entry["publication_year"],
                        "journal": entry["primary_location"]["source"]["display_name"],
                        "authors": current_authors
                    }
                    if entry_for_list not in relevant_papers:
                        relevant_papers.append(entry_for_list)
        except:
            print("FAIL:", entry)     
             
            # Inserting data into database
            # papers_col.add(
            #     documents="SENTENCE",
            #     metadatas={
            #         "title": "TITLE OF PAPER",
            #         "authors": json.dumps(["AUTHOR 1", "AUTHOR 2"]),
            #         "publication_year": "YEAR_OF_PUBLICATION",
            #         "journal_name": "JOURNAL_NAME",
            #         "place": "PLACE_OF_PUBLICATION"
            #     },
            #     ids="id_1"
            # )

## Connect the pull to MongoDB

In [14]:
all_entries_df1 = []
error = []

#@title Load Data from OpenAlex

#@markdown You can check out the list of concepts witht heir IDs [here](https://docs.google.com/spreadsheets/d/1LBFHjPt4rj_9r0t0TTAlT68NwOtNH8Z21lBMsJDMoZg/edit#gid=575855905), e.g., NLP c204321447 
# specify endpoint
endpoint = 'works'
concept = "'c144133560 '" #@param {type:"string"}
oa = False #@param {type:"boolean"}
nDocs = 50000 #@param {type:"slider", min:200, max:3000, step:1}
from_pub_date = "2000-01-01" #@param {type:"date"}
to_pub_date = '2023-04-01'
#@markdown Enter your email for API call to OpenAlex. It is not stored but just used for the API call to OpenAlex.
email = 'yasmine.sarraj@hotmail.com'#@param {type:"string"} 


def OA(oa):
  if True:
    return 'true'
  else:
    return 'false'

oa_str = OA(oa)

# build the 'filter' parameter
filters = ",".join((
    f'concepts.id:{concept}',
    'is_paratext:false', 
    f'from_publication_date:{from_pub_date}',
    f'to_publication_date:{to_pub_date}',
    f'is_oa:{oa_str}'
))

# put the URL together

filtered_works_url = f'https://api.openalex.org/{endpoint}?mailto={email}&filter={filters}'
print(f'complete URL with filters:\n{filtered_works_url}')

paging_param = 'per-page=100&cursor=*'

works_query = f'{filtered_works_url}&{paging_param}'

response = requests.get(works_query)
meta = json.loads(response.text)['meta']
next_cursor = meta['next_cursor']
save_to_chroma(json.loads(response.text)['results'])


cycles = math.floor((meta['count'] - 100) / meta['per_page'])+1
if cycles > 30:
  cycles = int(nDocs/100)

for cycle in range(cycles):
  cycle_query = f'{works_query[:-1]}{next_cursor}'
  response = requests.get(cycle_query)
  meta = json.loads(response.text)['meta']
  next_cursor = meta['next_cursor']
  save_to_chroma(json.loads(response.text)['results'])
  

complete URL with filters:
https://api.openalex.org/works?mailto=yasmine.sarraj@hotmail.com&filter=concepts.id:'c144133560 ',is_paratext:false,from_publication_date:2000-01-01,to_publication_date:2023-04-01,is_oa:true
FAIL: {'id': 'https://openalex.org/W2130950828', 'doi': 'https://doi.org/10.1086/586910', 'title': 'A Room with a Viewpoint: Using Social Norms to Motivate Environmental Conservation in Hotels', 'display_name': 'A Room with a Viewpoint: Using Social Norms to Motivate Environmental Conservation in Hotels', 'publication_year': 2008, 'publication_date': '2008-10-01', 'ids': {'openalex': 'https://openalex.org/W2130950828', 'doi': 'https://doi.org/10.1086/586910', 'mag': '2130950828'}, 'primary_location': {'is_oa': True, 'landing_page_url': 'https://doi.org/10.1086/586910', 'pdf_url': 'https://academic.oup.com/jcr/article-pdf/35/3/472/5251693/35-3-472.pdf', 'source': None, 'license': None, 'version': 'publishedVersion'}, 'host_venue': {'id': 'https://openalex.org/S145429826', 

KeyboardInterrupt: 

In [15]:
len(relevant_papers)

198

In [19]:
with open("papers.json", "r") as f:
    paper_data = json.load(f)

len(paper_data)

1145

In [17]:
for i in relevant_papers:
    if i not in paper_data:
        paper_data.append(i)

In [18]:
with open("papers.json", "w") as f:
    json.dump(paper_data, f, indent=2)